In [2]:
!git clone https://github.com/yysijie/st-gcn.git
%cd st-gcn


Cloning into 'st-gcn'...
remote: Enumerating objects: 1523, done.
remote: Total 1523 (delta 0), reused 0 (delta 0), pack-reused 1523 (from 1)
Receiving objects: 100% (1523/1523), 57.17 MiB | 24.97 MiB/s, done.
Resolving deltas: 100% (780/780), done.
/kaggle/working/st-gcn


In [3]:
# Install PyTorch (should already be available in Kaggle)
!pip install torch torchvision

# Install additional required libraries
!pip install -r requirements.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.4 MB/s eta 0:00:00:00:01


In [6]:
!mkdir -p models
!cp /kaggle/working/st-gcn/models/


cp: missing destination file operand after '/kaggle/working/st-gcn/models/'
Try 'cp --help' for more information.
